In [1]:
from io import open
import glob
import os
import unicodedata
import string
import pandas as pd
import random
random.seed(224)

In [2]:
def findFiles(path): 
    return glob.glob(path)

In [3]:
print(findFiles('surnames/names/*.txt'))

['surnames/names/Czech.txt', 'surnames/names/German.txt', 'surnames/names/Arabic.txt', 'surnames/names/Japanese.txt', 'surnames/names/Chinese.txt', 'surnames/names/Vietnamese.txt', 'surnames/names/Russian.txt', 'surnames/names/French.txt', 'surnames/names/Irish.txt', 'surnames/names/English.txt', 'surnames/names/Spanish.txt', 'surnames/names/Greek.txt', 'surnames/names/Italian.txt', 'surnames/names/Portuguese.txt', 'surnames/names/Scottish.txt', 'surnames/names/Dutch.txt', 'surnames/names/Korean.txt', 'surnames/names/Polish.txt']


In [4]:
all_letters = string.ascii_letters + " .,;'"
num_letters = len(all_letters)

In [5]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn' # "Mn" stands for Nonspacing_Mark, which is similar to unicodedata.combining 
        and c in all_letters
    )


#test 
print(unicodeToAscii('Ślusàrski'))

Slusarski


In [6]:
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('surnames/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

num_categories = len(all_categories)

In [7]:
print(all_categories)

['Czech', 'German', 'Arabic', 'Japanese', 'Chinese', 'Vietnamese', 'Russian', 'French', 'Irish', 'English', 'Spanish', 'Greek', 'Italian', 'Portuguese', 'Scottish', 'Dutch', 'Korean', 'Polish']


In [8]:
category_lines[all_categories[1]][:5] # first germany name

['Abbing', 'Abel', 'Abeln', 'Abt', 'Achilles']

In [9]:
num_names_percate = [len(category_lines[i]) for i in all_categories]

[519,
 724,
 2000,
 991,
 268,
 73,
 9408,
 277,
 232,
 3668,
 298,
 203,
 709,
 74,
 100,
 297,
 94,
 139]

In [10]:
names_country = []
for n in [(str(i + ' ') * j).split() for i,j in zip(all_categories,num_names_percate)]:
    for m in n:
        names_country.append(m)
        
names = []
for n in [category_lines[i] for i in all_categories]:
    for m in n:
        names.append(m)

In [11]:
len_ = len(names)
trainp =.7
validp = 0.15
testp =0.15
split = ['train'] * int(len_*trainp) + ['valid'] * int(len_*validp) + ['test'] * (int(len_*testp)+1)
random.shuffle(split)

In [12]:
surname_df = pd.DataFrame(data = {'names':names,'names_country':names_country,'split':split})
surname_df.head()


,names,names_country,split
0,Abl,Czech,valid
1,Adsit,Czech,test
2,Ajdrna,Czech,train
3,Alt,Czech,train
4,Antonowitsch,Czech,valid


In [13]:
#surname_df.to_csv('surnames/surname.csv')

# Modeling --- Online learning

In [18]:
import torch #爸爸！！！爸爸你好


def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, num_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, num_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('X'))

print(lineToTensor('Alex').size()) # 5 charaters and, for each letter it should be 1 X 57 

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([4, 1, 57])


# Buidling RNN  from nn.Linear

In [22]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size) # concate the t-1 hidden with t input data
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(num_letters, n_hidden, num_categories)
rnn

RNN(
  (i2h): Linear(in_features=185, out_features=128, bias=True)
  (i2o): Linear(in_features=185, out_features=18, bias=True)
  (softmax): LogSoftmax()
)

## testing

In [23]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.8438, -2.9200, -2.8530, -2.8901, -2.9252, -2.8329, -2.9083, -2.8293,
         -2.9120, -3.0044, -2.8805, -2.8611, -2.9719, -2.9716, -2.8852, -2.7895,
         -2.9125, -2.8615]], grad_fn=<LogSoftmaxBackward>)


In [24]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output)) # randomly initial outcome is Dutch.

('Dutch', 15)


In [25]:
#We wont use batch at all, this is an online learning example
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

In [29]:
import torch.optim as optim
criterion = nn.NLLLoss()
optimizer = optim.SGD(rnn.parameters(),lr= 0.01)

In [30]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):  # When didn't care about efficiency we dont have to pad our inputdata
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    optimizer.step()

    return output, loss.item()

In [ ]:
import time
import math

n_iters = 100000
print_every = 5000
plot_every = 1000


# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 13s) 2.4724 Pietsuh / Portuguese ✗ (Russian)
